In [8]:
import pandas as pd
import sys
from __globals import glob
from _utilities import prepare_data, prepare_database
sys.path.append("../tools/")
import _db_tools as db #Personal tool for managing sqlite databases in data science

####Helper Functions####
def limits_gen(df, means, *argv):
    '''Generate the limit values for a list containing the means in a DataFrame.
    Calculates the total mean for each fiber axis and applies it to the corresponding rows.
    Supports common (one) tolerances and single (low and high) tolerances.
    Parameters:
    df (pd.DataFrame): The MEAS dataframe to get its size.
    means (list): A list of means to generate limits for.
    *argv: Variable-length arguments for tolerances. Can be one or two values.
    Returns:
    pd.DataFrame: A DataFrame containing the generated limits.
    '''
    if len(argv) == 1: 
        l_tolerance = argv[0]
        h_tolerance = argv[0]
    elif len(argv) == 2:
        l_tolerance = argv[0]
        h_tolerance = argv[1]
    else:
        raise ValueError("Invalid number of tolerance values. Must be 1 or 2.")
    limits = pd.DataFrame(columns=["LO_LIMIT", "HI_LIMIT"])
    for index in range(int(df.shape[0])):
        if index % 2 == 0:
            low_limit = round(means[0] - h_tolerance, 4)
            high_limit = round(means[0] + l_tolerance, 4)
        else:
            low_limit = round(means[1] - h_tolerance, 4)
            high_limit = round(means[1] + l_tolerance, 4)
        current_limits_df = pd.DataFrame({"LO_LIMIT": [low_limit], "HI_LIMIT": [high_limit]}) #Create a DataFrame with the current low_limit and high_limit values
        limits = pd.concat([limits, current_limits_df], ignore_index=True, axis=0) #Concatenate the current limits DataFrame with the main 'limits' DataFrame
    return limits


In [9]:
#Data preparation
dbh = db.SQLite_Data_Extractor("database.db") #Connect to the database
MEAS = dbh.retrieve(glob.tooling) #Get the desired tooling data
dbh.close_conn() 

../database/database.db found.
Table PASSAT_B9 retrieved succesfully.
Closed connection to: ../database/database.db


In [10]:
#Calculation of the absolute means for fiber x and fiber y
resume = MEAS.transpose().describe() #Transpose the df first due to describe() working in columns.
rough_means = list(resume.iloc[1, :].values)
means = []; means_fbx = []; means_fby = [] #Preallocation
for i, mean in enumerate(rough_means): #Iterates and rounds every mean value
    mean = round(mean, 4)
    means_fbx.append(mean) if i % 2 == 0 else means_fby.append(mean)
    means.append(mean)
abs_mean_fbx = sum(means_fbx) / len(means_fbx)
abs_mean_fby = sum(means_fby) / len(means_fby)
means = [abs_mean_fbx, abs_mean_fby]
print("Means:") 
print("Fiber x: " + str(round(abs_mean_fbx, 4)))
print("Fiber y: " + str(round(abs_mean_fby, 4)))



Means:
Fiber x: 0.3313
Fiber y: 0.3555


In [11]:
LIMITS = limits_gen(MEAS, means, glob.tolerance)
LIMITS

,LO_LIMIT,HI_LIMIT
0,0.3163,0.3463
1,0.3405,0.3705
2,0.3163,0.3463
3,0.3405,0.3705
4,0.3163,0.3463
5,0.3405,0.3705
6,0.3163,0.3463
7,0.3405,0.3705
8,0.3163,0.3463
9,0.3405,0.3705


In [12]:
##Creation of the target dataframe with correct limits
new_column_names = [f'{i+1}' for i in range(len(MEAS.columns))] #Columns renaming
MEAS.columns = new_column_names
new_row_names = []; counter = 1; fiber = "X" #Preallocation
for i in range(0, len(df)): #Rows renaming
    new_row_name = f'Guia_Luz_Blanco_FB{counter}_{fiber}' 
    new_row_names.append(new_row_name)
    if i % 2 == 1: #Alternate the fiber axis and update the fober number every two fibers
        counter += 1
        fiber = "X"
    else:
        fiber = "Y"
MEAS.index = new_row_names
LIMITS.index = new_row_names
target = pd.concat([MEAS, LIMITS], axis=1) #Concatenates the measures and limits
target

,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,LO_LIMIT,HI_LIMIT
Guia_Luz_Blanco_FB1_X,0.3326,0.3326,0.3326,0.3326,0.3327,0.3326,0.3326,0.3326,0.3327,0.3327,...,0.3327,0.3327,0.3327,0.3327,0.3324,0.3324,0.3326,0.3326,0.3163,0.3463
Guia_Luz_Blanco_FB1_Y,0.3557,0.3557,0.3557,0.3557,0.3557,0.3557,0.3557,0.3557,0.3557,0.3557,...,0.3557,0.3557,0.3557,0.3557,0.3556,0.3556,0.3557,0.3557,0.3405,0.3705
Guia_Luz_Blanco_FB2_X,0.3325,0.3324,0.3324,0.3324,0.3323,0.3324,0.3323,0.3323,0.3323,0.3324,...,0.3324,0.3323,0.3324,0.3323,0.3323,0.3323,0.3323,0.3323,0.3163,0.3463
Guia_Luz_Blanco_FB2_Y,0.3573,0.3573,0.3573,0.3573,0.3572,0.3572,0.3572,0.3572,0.3572,0.3573,...,0.3572,0.3571,0.3571,0.3572,0.3572,0.3571,0.3572,0.3571,0.3405,0.3705
Guia_Luz_Blanco_FB3_X,0.3420,0.3422,0.3420,0.3420,0.3420,0.3419,0.3420,0.3420,0.3419,0.3419,...,0.3418,0.3418,0.3418,0.3419,0.3419,0.3419,0.3419,0.3419,0.3163,0.3463
Guia_Luz_Blanco_FB3_Y,0.3628,0.3630,0.3629,0.3629,0.3629,0.3629,0.3629,0.3628,0.3629,0.3628,...,0.3628,0.3628,0.3628,0.3628,0.3628,0.3628,0.3628,0.3628,0.3405,0.3705
Guia_Luz_Blanco_FB4_X,0.3318,0.3317,0.3316,0.3316,0.3316,0.3317,0.3317,0.3316,0.3316,0.3317,...,0.3316,0.3316,0.3316,0.3317,0.3317,0.3317,0.3317,0.3316,0.3163,0.3463
Guia_Luz_Blanco_FB4_Y,0.3572,0.3571,0.3571,0.3571,0.3571,0.3571,0.3571,0.3571,0.3571,0.3571,...,0.3570,0.3571,0.3571,0.3571,0.3571,0.3571,0.3571,0.3570,0.3405,0.3705
Guia_Luz_Blanco_FB5_X,0.3262,0.3258,0.3259,0.3258,0.3259,0.3259,0.3258,0.3258,0.3259,0.3259,...,0.3259,0.3259,0.3259,0.3259,0.3260,0.3260,0.3260,0.3259,0.3163,0.3463
Guia_Luz_Blanco_FB5_Y,0.3526,0.3525,0.3525,0.3524,0.3524,0.3525,0.3524,0.3524,0.3524,0.3524,...,0.3524,0.3524,0.3524,0.3524,0.3524,0.3524,0.3525,0.3524,0.3405,0.3705


In [13]:
##Data export
confirmation = input("Do you want to export the data to a new Target.xlsx file? (y/n): ").strip().lower()
if confirmation == 'y':
    target.to_excel('../a2_output/Target_with_limits.xlsx', index=True, header=True)
    #os.startfile('../a2_output/Target_with_limits.xlsx')
    print('Data exported in: ../a2_output/Target_with_limits.xlsx')
else:
    print("Operation canceled.")

Operation canceled.


In [14]:
##Update of the database table
confirmation = input("Do you want to export the new limits to your database (y/n): ").strip().lower()
if confirmation == 'y':
    prepare_database(LIMITS, glob.tooling+"_limits") #Store a df inside the database of the project
else:
    print("Operation canceled.")

Operation canceled.
